In [96]:
import pandas as pd
import numpy as np
import re
import json
import ast

### 1.0 Lectura de archivos

In [97]:
df = pd.read_csv("../movies_dataset.csv")
print("dimensiones de la data:",df.shape)

dimensiones de la data: (45466, 24)


C:\Users\Pablo\AppData\Local\Temp\ipykernel_12092\2506349904.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../movies_dataset.csv")


### 1.1 valores nulos de la data

In [98]:
df.isna().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [99]:
#df_peliculas['release_date'] = pd.to_datetime(df_peliculas['release_date'], errors='coerce').dt.strftime('%Y-%m-%d')

#### 1.2 Tratamiento sobre columnas que sean colecciones

In [100]:
df_diccts = df[['belongs_to_collection','genres','production_companies','production_countries','spoken_languages']] 

In [101]:
df_diccts.sample(3)

,belongs_to_collection,genres,production_companies,production_countries,spoken_languages
3075,NaN,"[{'id': 18, 'name': 'Drama'}]",[],[],"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]"
38628,NaN,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'name': 'Minerva Film SpA', 'id': 153}, {'na...","[{'iso_3166_1': 'MK', 'name': 'Macedonia'}, {'...","[{'iso_639_1': 'sq', 'name': 'shqip'}, {'iso_6..."
28678,NaN,"[{'id': 18, 'name': 'Drama'}]","[{'name': 'Warner Brothers/Seven Arts', 'id': ...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]"


In [102]:
from Funciones import convertir_dicc_to_list #para estructuras como belong_to_collection
from Funciones import convertir_list_to_dicct # para estructuras como genres
from Funciones import convertir_str_list # para estructuras como spoken_lenguage

In [103]:
col1 =df['belongs_to_collection']
col2 =df['genres']
col3 =df['production_companies']
col4 =df['production_countries']
col5 =df['spoken_languages']

col1 = convertir_dicc_to_list(col1)
col2 = convertir_list_to_dicct(col2)
col3 = convertir_list_to_dicct(col3)
col4 = convertir_list_to_dicct(col4)
col5 = convertir_str_list(col5)


In [104]:
#reemplazar los valores de las columnas por estas nuevas listas
df['belongs_to_collection'] = pd.Series(col1)
df['genres'] = pd.Series(col2)
df['production_companies'] = pd.Series(col3)
df['production_countries'] = pd.Series(col4)
df['spoken_languages'] = pd.Series(col5)

In [105]:
df[['belongs_to_collection','genres','production_companies','production_countries','spoken_languages']].sample(3)

,belongs_to_collection,genres,production_companies,production_countries,spoken_languages
36935,NaN,"[[10749, Romance], [12, Adventure], [18, Drama]]",NaN,NaN,NaN
24084,NaN,"[[53, Thriller], [18, Drama]]",NaN,NaN,NaN
8037,NaN,[],[],[],"[[iso_639_1, en], [name, English]]"


### 1.3 Requisitos de procesamiento de los datos

#### requisito: Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

In [106]:
df['revenue'].fillna(0,inplace=True)
df['budget'].fillna(0,inplace=True)

#### requisito: Los valores nulos del campo release date deben eliminarse.

In [107]:
df = df.dropna(axis=0,subset='release_date')

#### requisito: De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [108]:
df['release_date'] = pd.to_datetime(df['release_date'],errors='coerce')
df['release_year'] = df['release_date'].dt.year

#### requisito: Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [109]:
df['budget'] = pd.to_numeric(df['budget'],errors='coerce')
df['return'] = np.where((df['budget'].isna() | df['revenue'].isna() | (df['budget'] == 0)),
                     0, df['revenue'] / df['budget'])

#### requisito: Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

In [110]:
df = df.drop(columns= ['video','imdb_id','adult','original_title','poster_path','homepage'])

#### requisito: columna release_year con el año

In [111]:
df['release_year'] = df['release_date'].dt.year

### segundo dataset: credits

In [112]:
df_creditos = pd.read_csv("../credits.csv")

In [113]:
df_creditos.head(3)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602


### creacion de dataframes individuales para cast y crew

#### df cast

In [114]:
linea_cast = df_creditos['cast']
linea_id = df_creditos['id']
datos_cast = {}
datos_cast["id_credit"] = []
datos_cast["cast_id"] = []
datos_cast["character"] = []
datos_cast["credit_id"] = []
datos_cast["gender"] = []
datos_cast["id"] = []
datos_cast["name"] = []
datos_cast["order"] = []
datos_cast["profile_path"] = []
try:
    for idx, lis in enumerate(linea_cast):
        linea = ast.literal_eval(lis)
        id_actual = linea_id.iloc[idx]
        for dic in linea:
            datos_cast['id_credit'].append(id_actual)
            for clave,valor in dic.items():
                datos_cast[clave].append(valor)
except:
    pass

datos_cast = pd.DataFrame(datos_cast)

print(datos_cast.shape)
datos_cast.head()

(562474, 9)


,id_credit,cast_id,character,credit_id,gender,id,name,order,profile_path
0,862,14,Woody (voice),52fe4284c3a36847f8024f95,2,31,Tom Hanks,0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
1,862,15,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2,12898,Tim Allen,1,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
2,862,16,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2,7167,Don Rickles,2,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
3,862,17,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2,12899,Jim Varney,3,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
4,862,18,Rex (voice),52fe4284c3a36847f8024fa5,2,12900,Wallace Shawn,4,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg


#### df crew

In [115]:
linea_crew = df_creditos['crew']
linea_id = df_creditos['id']

datos_crew = {}
datos_crew["id_credit"] = []
datos_crew["credit_id"] = []
datos_crew["department"] = []
datos_crew["gender"] = []
datos_crew["id"] = []
datos_crew["job"] = []
datos_crew["name"] = []
datos_crew["profile_path"] = []
try:
    for idx, lis in enumerate(linea_crew):
        linea = ast.literal_eval(lis)
        id_actual = linea_id.iloc[idx]
        for dic in linea:
            datos_crew["id_credit"].append(id_actual)  # Agrega el id original repetido
            for clave, valor in dic.items():
                datos_crew[clave].append(valor)
except Exception as e:
    print(f"Ocurrió un error: {e}")

# Convertir a DataFrame para verificar los resultados
datos_crew = pd.DataFrame(datos_crew)
datos_crew.head()

,id_credit,credit_id,department,gender,id,job,name,profile_path
0,862,52fe4284c3a36847f8024f49,Directing,2,7879,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
1,862,52fe4284c3a36847f8024f4f,Writing,2,12891,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
2,862,52fe4284c3a36847f8024f55,Writing,2,7,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
3,862,52fe4284c3a36847f8024f5b,Writing,2,12892,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
4,862,52fe4284c3a36847f8024f61,Writing,0,12893,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg


#### eliminacion de columnas sin interes

In [116]:
df= df.drop(columns= ['tagline'])
datos_cast = datos_cast.drop(columns=['profile_path'])
datos_crew = datos_crew.drop(columns=['profile_path'])
#eliminacion de nans
#df.dropna(axis=0,inplace=True)
#datos_crew.dropna(axis=0,inplace=True)
#datos_cast.dropna(axis=0,inplace=True)
print(df.shape)
print(datos_crew.shape)
print(datos_cast.shape)

(45379, 19)
(464314, 7)
(562474, 8)


#### Checkeos de tipos de datos

In [117]:
df['id'] = pd.to_numeric(df['id'],errors='coerce')
df['popularity'] = pd.to_numeric(df['popularity'],errors='coerce')

#### Crear datasets ya limpios

In [118]:
#datos_cast.to_csv('df_cast.csv',index=False)
#datos_crew.to_csv('df_crew.csv',index=False)
#df.to_csv('movies.csv',index=False)

## Finalizacion de ETL

In [122]:
"""
lista_id = []
lista_titulos = []
for coleccion in df['belongs_to_collection']:
    try:
        #obtener los valores de los ids de las peliculas
        #print(x[0][1])
        #print(coleccion[0][1])
        #print(coleccion[1][1])
        if int(coleccion[0][1]) not in lista_id:
            lista_id.append(coleccion[0][1])
        if str(coleccion[1][1]) not in lista_titulos:
            lista_titulos.append(coleccion[1][1])
    except:
        pass

lis_peliculas = pd.DataFrame(data={"id":lista_id,
                                   "titulo":lista_titulos})

lis_peliculas['titulo']
"""

'\nlista_id = []\nlista_titulos = []\nfor coleccion in df[\'belongs_to_collection\']:\n    try:\n        #obtener los valores de los ids de las peliculas\n        #print(x[0][1])\n        #print(coleccion[0][1])\n        #print(coleccion[1][1])\n        if int(coleccion[0][1]) not in lista_id:\n            lista_id.append(coleccion[0][1])\n        if str(coleccion[1][1]) not in lista_titulos:\n            lista_titulos.append(coleccion[1][1])\n    except:\n        pass\n\nlis_peliculas = pd.DataFrame(data={"id":lista_id,\n                                   "titulo":lista_titulos})\n\nlis_peliculas[\'titulo\']\n'

In [123]:
"""
#### Expansion de columnas
def expandcolumns(df, columns):
    for column in columns:
        df[column] = df[column].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else []) #Convierte el contenido de cada celda en la columna column de df de una cadena de texto que representa una lista de diccionarios a una lista de diccionarios Python, utilizando ast.literal_eval. Si el valor es nulo (pd.notnull(x)), se devuelve una lista vacía.
        # Expande la columna column en filas, de modo que cada elemento de la lista de diccionarios se convierte en una fila separada en df
        df = df.explode(column)
        # Normaliza la columna column después de expandirla, lo que significa que convierte cada diccionario dentro de la lista en una serie de columnas.
        col_df = pd.json_normalize(df[column])
        # Renombra las columnas normalizadas agregando el nombre original de la columna como prefijo, para evitar conflictos si hay nombres de columnas duplicados en diferentes listas de diccionarios.
        col_df = col_df.add_prefix(f'{column}')
        # Elimina la columna original column del DataFrame df y la reemplaza con las columnas normalizadas (col_df), asegurándose de que el índice del DataFrame se reinicie para evitar problemas con el índice de las filas.
        df = df.drop(columns=[column]).reset_index(drop=True).join(col_df)
    return df

columns_to_expand = ['cast', 'crew']


df_credit_norm = expandcolumns(df_creditos, columns_to_expand)



#leer el archivo y expandir columnas individualmente
df_creditos = pd.read_csv("../credits.csv")
print(df_creditos.shape)
print(df_creditos.columns)
df_creditos_cast = expandcolumns(df_creditos, ['cast'])
df_creditos_crew = expandcolumns(df_creditos, ['crew'])
print(df_creditos_crew.shape)
print(df_creditos_cast.shape)
df_creditos_cast = df_creditos_cast.drop(columns='crew')
df_creditos_crew = df_creditos_crew.drop(columns='cast')
df_creditos_cast

"""

'\n#### Expansion de columnas\ndef expandcolumns(df, columns):\n    for column in columns:\n        df[column] = df[column].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else []) #Convierte el contenido de cada celda en la columna column de df de una cadena de texto que representa una lista de diccionarios a una lista de diccionarios Python, utilizando ast.literal_eval. Si el valor es nulo (pd.notnull(x)), se devuelve una lista vacía.\n        # Expande la columna column en filas, de modo que cada elemento de la lista de diccionarios se convierte en una fila separada en df\n        df = df.explode(column)\n        # Normaliza la columna column después de expandirla, lo que significa que convierte cada diccionario dentro de la lista en una serie de columnas.\n        col_df = pd.json_normalize(df[column])\n        # Renombra las columnas normalizadas agregando el nombre original de la columna como prefijo, para evitar conflictos si hay nombres de columnas duplicados en diferentes

In [124]:
"""
# una opcion es crear un dataframe donde estaran las peliculas de enero por ejemplo
#january_movies = df_unico[df_unico['release_date'].dt.month == 1]
# una alternativa mas eficiente seria la siguiente:
cantidad_peliculas_mes = df_unico['release_date'].dt.month.value_counts()
cantidad_peliculas_mes.sort_index(inplace=True)
# donde guardo el valor de peliculas estrenadas en ese mes
#luego de esto podemos guardar el numero en una variable
# considerar que los corchetes de este metodo, comienzan a contar desde 1 y no desde 0, siendo 1 = enero.
cantidad_peliculas_mes_enero = df_unico['release_date'].dt.month.value_counts()[1]
#tambien accediendo a los elementos de la seria ya creada.
enero = cantidad_peliculas_mes[1]
print(cantidad_peliculas_mes)
print(cantidad_peliculas_mes_enero)
print(enero)
"""

"\n# una opcion es crear un dataframe donde estaran las peliculas de enero por ejemplo\n#january_movies = df_unico[df_unico['release_date'].dt.month == 1]\n# una alternativa mas eficiente seria la siguiente:\ncantidad_peliculas_mes = df_unico['release_date'].dt.month.value_counts()\ncantidad_peliculas_mes.sort_index(inplace=True)\n# donde guardo el valor de peliculas estrenadas en ese mes\n#luego de esto podemos guardar el numero en una variable\n# considerar que los corchetes de este metodo, comienzan a contar desde 1 y no desde 0, siendo 1 = enero.\ncantidad_peliculas_mes_enero = df_unico['release_date'].dt.month.value_counts()[1]\n#tambien accediendo a los elementos de la seria ya creada.\nenero = cantidad_peliculas_mes[1]\nprint(cantidad_peliculas_mes)\nprint(cantidad_peliculas_mes_enero)\nprint(enero)\n"

In [125]:
#df_creditos = pd.merge(df_creditos_cast, df_creditos_crew, on='id',how='outer')
#unique_ids_count_df = pd.merge(unique_ids_count,df['release_date'],on='id',how='left')

codigo para no se que

Lista de valores específicos que quieres eliminar
specific_values = ['1997-08-20', '2012-09-29', '2014-01-01']

Filtrar las filas que contienen cualquiera de los valores específicos en cualquier columna
filas_borrar = df_movies[df_movies.applymap(lambda x: str(x) in specific_values).any(axis=1)].index

Eliminar las filas filtradas
df_movies.drop(filas_borrar, inplace=True)

In [126]:
#### obtener un dataset con los valores unicos de genero

"""
lis_dic = []
for x in df['generes']:
    valor_dic = ast.literal_eval(x)
    lis_dic.append(valor_dic)
    #diccionario = ast.literal_eval(diccionario_str)

df_generos = []
df_generos_id = []
for x in lis_dic:
    for y in x:
        if y['id'] not in df_generos_id:
            df_generos.append(y['name'])
            df_generos_id.append(y['id'])
        else:
            pass
dic_gen = {'id':df_generos_id,
           'genero':df_generos}
df_gen = pd.DataFrame(data=dic_gen)
df_gen
"""

"\nlis_dic = []\nfor x in df['generes']:\n    valor_dic = ast.literal_eval(x)\n    lis_dic.append(valor_dic)\n    #diccionario = ast.literal_eval(diccionario_str)\n\ndf_generos = []\ndf_generos_id = []\nfor x in lis_dic:\n    for y in x:\n        if y['id'] not in df_generos_id:\n            df_generos.append(y['name'])\n            df_generos_id.append(y['id'])\n        else:\n            pass\ndic_gen = {'id':df_generos_id,\n           'genero':df_generos}\ndf_gen = pd.DataFrame(data=dic_gen)\ndf_gen\n"

In [127]:
"""
return = []
for index, row in dftesting.iterrows():
    if row[1] != 0:
        return.append(row[10] / row[1])
    else:
        return.append(0)
return


todrop = [19730, 29503, 35587]
dfmovies_etl = dfmovies_etl.drop(todrop, axis=0)
"""

'\nreturn = []\nfor index, row in dftesting.iterrows():\n    if row[1] != 0:\n        return.append(row[10] / row[1])\n    else:\n        return.append(0)\nreturn\n\n\ntodrop = [19730, 29503, 35587]\ndfmovies_etl = dfmovies_etl.drop(todrop, axis=0)\n'